In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Read data and processing

In [2]:
data = pd.read_csv('Stars.csv')
data.head()

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0


In [3]:
data['Type'].unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [4]:
# label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in data.columns:
    if data[i].dtype == 'object':
        data[i] = le.fit_transform(data[i])

In [5]:
data.head()

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,8,5,0
1,3042,0.000500,0.1542,16.60,8,5,0
2,2600,0.000300,0.1020,18.70,8,5,0
3,2800,0.000200,0.1600,16.65,8,5,0
4,1939,0.000138,0.1030,20.06,8,5,0


## Training

### Extract feature

In [66]:
X,y = data.drop('Type',axis=1),data['Type']

In [67]:
X.shape

(240, 6)

### Using LDA to reduce dimensions

In [75]:
class LDA:
    '''
    Inverse transform

    Atributes:
    X_transformed: numpy array, shape = [n_samples,n_components]
    Returns:
    X_projected: numpy array, shape = [n_samples,n_features]
    
    '''
    def __init__(self,n_components):
      '''
        Parameters:
        n_components: int, default = 2

      '''
      self.n_components = n_components
      self.linear_discriminants = None
    def fit(self,X,y):

      '''
      Parameters:
      X: numpy array, shape = [n_samples,n_features]
      y: numpy array, shape = [n_samples,]
      '''
      n_features = X.shape[1]
      class_lables = np.unique(y)

      # S_W , S_B
      mean_overall = np.mean(X,axis = 0)
      S_W = np.zeros((n_features,n_features))
      S_B = np.zeros((n_features,n_features))

      for c in class_lables:
        X_c = X[y == c]
        mean_c = np.mean(X_c,axis = 0)
        # 4,4
        S_W += (X_c - mean_c).T.dot(X_c - mean_c)
        n_c = X_c.shape[0]
        mean_diff = (mean_c - mean_overall).reshape(n_features,1)
        S_B += n_c * (mean_diff).dot(mean_diff.T)
      A = np.linalg.inv(S_W).dot(S_B)
      # Same PCA
      eigenvalues,eigenveactors = np.linalg.eig(A)
      eigenveactors = eigenveactors.T
      indexs = np.argsort(abs(eigenvalues))[::-1]
      eigenvalues = eigenvalues[indexs]
      eigenvectors = eigenveactors[indexs]
      self.linear_discriminants = eigenvectors[0:self.n_components]

    def transform(self,X):

      '''
      Parameters:
      X: numpy array, shape = [n_samples,n_features]
      '''
      return np.dot(X,self.linear_discriminants.T)

In [69]:
lda = LDA(n_components=4)
lda.fit(X.to_numpy(),y)
X = lda.transform(X.to_numpy())

In [71]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

### Using Navies Bayes

In [73]:
import numpy as np


class NaiveBayes:

    '''
    Inverse transform

    Atributes:
    X_transformed: numpy array, shape = [n_samples,n_components]
    Returns:
    X_projected: numpy array, shape = [n_samples,n_features]

    '''
    def fit(self, X, y):

        '''
        Parameters:
        X: numpy array, shape = [n_samples,n_features]
        y: numpy array, shape = [n_samples,]
        '''
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y == c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        '''
        Parameters:
        X: numpy array, shape = [n_samples,n_features]
        
        Returns:
        y_pred: numpy array, shape = [n_samples,]
        '''
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        '''
        Parameters:
        class_idx: int
        x: numpy array, shape = [n_features,]

        Returns:
        pdf: numpy array, shape = [n_features,]

        '''
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
    
    def accuracy(self,y_true, y_pred):
        '''
        Parameters:
        y_true: numpy array, shape = [n_samples,]
        y_pred: numpy array, shape = [n_samples,]

        Return:
        accuracy: float
        '''
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

In [74]:
# Using Naive Bayes
nb = NaiveBayes()
nb.fit(X_train,y_train)
y_pred = nb.predict(X_test)
nb.accuracy(y_test,y_pred)

0.9791666666666666